<a href="https://colab.research.google.com/github/Waleed-Mujahid/CAO-project/blob/main/Cao_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# including dependecies
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [ ]:
# image dimensions, set as per need
img_width, img_height = 48,48  # 96 x 96 are suitable for micro-controllers

In [ ]:
# Loading our dataset from Drive
# First mount drive

train_data_dir = '/content/drive/MyDrive/faceData/train'
validation_data_dir = '/content/drive/MyDrive/faceData/test'

In [ ]:
# Data pre-processing
train_datagen = ImageDataGenerator( 
rescale=1. / 255,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True)

In [ ]:
# rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# Getting data generator from directory

train_generator = train_datagen.flow_from_directory(
train_data_dir,
batch_size = 16,
target_size=(img_width, img_height),
class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
batch_size = 21,
target_size=(img_width, img_height),
class_mode='binary')

In [ ]:
# Creating our model with Keras

model = Sequential()
model.add(Conv2D(3, (12, 12),activation='relu', input_shape=(img_width, img_height,3)))  # Input size of all images must be standard for model
model.add(MaxPooling2D(strides=2))

# model.add(Conv2D(10, (5, 5), activation='relu'))
# model.add(MaxPooling2D(strides=2))

model.add(Conv2D(6, (12, 12), activation='relu'))
model.add(MaxPooling2D(strides=2))

# model.add(Conv2D(30, (3, 3), activation='relu'))
# model.add(MaxPooling2D(strides=2))

model.add(Conv2D(12, (1, 1), activation='relu'))
model.add(MaxPooling2D(strides=2))

model.add(Flatten())
model.add(Dropout(0.3))
#model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0003),
metrics=['accuracy'])

In [ ]:
model.fit(
train_generator,
epochs=5,
validation_data=validation_generator) 

In [ ]:
model.evaluate(validation_generator)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt 
# Save image in set directory
# Read RGB image
#path = "/content/drive/MyDrive/faceData/test/true/WIN_20220513_00_40_18_Pro.jpg"
path = '/content/drive/MyDrive/faceData/test/false/WIN_20220513_00_40_45_Pro.jpg'
_img = Image.open(path)
_img.load()
plt.imshow(_img)

In [ ]:
img = tf.io.read_file(path)
img = tf.image.decode_image(img)
test_img = tf.image.resize(img, size = [img_height, img_height])
test_img = test_img/255.
val = model.predict(tf.expand_dims(test_img,axis=0))
print(val)

In [ ]:
model.save('myModel')

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('/content/myModel') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('cao.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
#!apt-get -qq install xxd
!xxd -i cao.tflite > waleed_FaceRecog_alt.cc

In [ ]:
# Print the signatures from the converted model
interpreter = tf.lite.Interpreter(model_content=tflite_model)
signatures = interpreter.get_signature_list()
print(signatures)